# Importing Libraries
    Pandas , Numpy , Matplotlib


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pathlib
import cv2
import pickle

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


In [ ]:
from torchsummary import summary

**Reading the first image in the image folder of the dataset**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
    

In [ ]:
single_img = plt.imread("/kaggle/input/data/images_001/images/00000001_000.png") 

**Checking the visual of the image**

In [ ]:
plt.imshow(single_img,cmap = "gray")

In [ ]:
data_entries = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")

In [ ]:
data_entries

In [ ]:
all_columns = data_entries.columns
all_columns = list(all_columns)

In [ ]:
useful_columns = all_columns[0:2] + all_columns[7:11]

In [ ]:
useful_data = data_entries[useful_columns]
useful_data

In [ ]:
useful_data["Image Path"] = "Image path"

In [ ]:
Image_path_mapping = []

for i in range(1,13):
    folder_path = f"/kaggle/input/data/images_{i:03}/images"
    
    for abs_path in pathlib.Path(folder_path).glob("*.png"):

        path_file_name = abs_path.name
        Image_path_mapping.append((path_file_name,str(abs_path)))

path_dict = dict(Image_path_mapping)

useful_data.loc[:, "Image Path"] = useful_data["Image Index"].map(path_dict)

In [ ]:
pd.set_option('display.max_colwidth', None)
useful_data

In [ ]:
unique_diseases = set()

for disease in useful_data["Finding Labels"]:
    unique_diseases.update(set(disease.split("|")))

In [ ]:
unique_diseases = list(unique_diseases)
unique_diseases.remove("No Finding")

In [ ]:
unique_diseases

In [ ]:
diseases_to_idx = dict(zip(unique_diseases , range(len(unique_diseases))))

In [ ]:
diseases_to_idx

In [ ]:
"""with open ("diseases_to_idx", "wb") as file_handle:
    pickle.dump(diseases_to_idx,file_handle)"""

In [ ]:
BBox_list = pd.read_csv("/kaggle/input/data/BBox_List_2017.csv")

In [ ]:
BBox_list

In [ ]:
test_column = "Image Index"

In [ ]:
testing_data  = useful_data[useful_data[test_column].isin(BBox_list[test_column])]
training_data = useful_data[~useful_data[test_column].isin(BBox_list[test_column])]

In [ ]:
training_data

In [ ]:
training_data

In [ ]:
useful_data.shape

In [ ]:
BBox_list.shape

In [ ]:
missing_ids = BBox_list[~BBox_list[test_column].isin(useful_data[test_column])]
print(f"Missing from useful_data: {len(missing_ids)}")
display(missing_ids.head())


In [ ]:
common_ids = BBox_list[BBox_list[test_column].isin(useful_data[test_column])]
print(f"Common: {len(common_ids)}")  


In [ ]:
print("Unique images in BBox_list:", BBox_list[test_column].nunique())
print("Unique images in testing_data:", testing_data[test_column].nunique())


In [ ]:
print(useful_data[test_column].value_counts())
print(testing_data[test_column].value_counts())


In [ ]:
def get_multi_hot_label_vector(provided_data):
    

    vector = []


    for single_img_labels in provided_data["Finding Labels"]:
    
        diseases = single_img_labels.split("|")
        single_img_multi_hot_vector = np.zeros((len(diseases_to_idx),))

        if "No Finding" not in diseases:
        
            for single_disease in diseases:
                single_img_multi_hot_vector[diseases_to_idx[single_disease]] = 1.0

        vector.append(single_img_multi_hot_vector)

    return np.array(vector)
        

In [ ]:
y_train = get_multi_hot_label_vector(training_data)
y_test = get_multi_hot_label_vector(testing_data)

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
y_test.shape

In [ ]:
y_test

In [ ]:
pd.reset_option('display.max_colwidth')


In [ ]:
def testing_data_mini_batches_generator(Bounding_box_list,testing_data_df,mb_size=10):

    complete_testing_bbox_df = Bounding_box_list.merge(testing_data_df, on="Image Index", how="inner")
    

    for i in range(complete_testing_bbox_df.shape[0]//mb_size):

        img_np_arr_mb_list = list()
        label_mb_list = list()
        bbox_mb_list = list

        for j in range(1*mb_size,(i+1)*mb_size):
            row = complete_testing_bbox_df.iloc[j]

            single_img_path = row["Image Path"]
            img_np_arr = plt.imread(single_img_path)

            original_img_width = row["OriginalImage[Width"]
            original_img_height = row["Height]"]

            resized_img_np_arr = cv2.resize(img_np_arr,(224,224))
            three_channel_np_arr = cv2.cvtColor(resized_img_np_arr,cv2.COLOR_GRAY2RGB)


            x,y,w,h = row["[x"],row["y"],row["w"],row["h]"]
            scale_x = 224.0/original_img_width
            scale_y = 224.0/original_img_height
            bbox_scaled_acc = [x*scale_x,y*scale_y,w*scale_x,h*scale_y]

            bbox_mb_list.append(bbox_scaled_acc)
            img_np_arr_mb_list.append(three_channel_np_arr)

        x_test_mb = np.array(img_np_arr)
        y_test_mb = y_test[i*mb_size:(i+1)*mb_size,:]
        bboxes_mb = np.array(bbox_mb_list)

        yield x_test_mb, y_test_mb, bboxes_mb 
            

In [ ]:
def training_data_mini_batches_generator(training_data_df, mb_size=10):
    
    for i in range (training_data_df.shape[0]//mb_size):

        img_np_array_mb_list = list()
        
        for j in range(i*mb_size,(i+1)*mb_size):
            
            single_img_path  = training_data_df["Image Path"].iloc[j]
            img_np_array = plt.imread(single_img_path)

            resized_img_np_array = cv2.resize(img_np_array,(224,224))

            if len(resized_img_np_array.shape) == 2 or resized_img_np_array.shape[2] == 1:
            # grayscale image (H, W) or (H, W, 1)
                three_channel_np_array = cv2.cvtColor(resized_img_np_array, cv2.COLOR_GRAY2RGB)
            elif resized_img_np_array.shape[2] == 4:
            # image with 4 channels (e.g., RGBA) → drop alpha
                three_channel_np_array = resized_img_np_array[:, :, :3]
            else:
            # already 3 channels
                three_channel_np_array = resized_img_np_array


            img_np_array_mb_list.append(three_channel_np_array)

        x_train_mb = np.array(img_np_array_mb_list)
        y_train_mb = y_train[i*mb_size:(i+1)*mb_size,:]

        yield x_train_mb,y_train_mb

In [ ]:
class our_custom_resnet50_cnn(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.resnet50_full_net = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

        for param in self.resnet50_full_net.parameters():
            param.requires_grad = False

        self.resnet50_full_net.fc = nn.Sequential(
            nn.Linear(self.resnet50_full_net.fc.in_features, num_classes),
            nn.Sigmoid()  
        )

    def forward(self, x):
        
        y_pred_mb =  self.resnet50_full_net(x)
        return y_pred_mb

In [ ]:
"""class our_custom_resnet50_cnn(torch.nn.Module):

    def __init__(self,num_classes):
        super().__init__()

        self.resnet50_full_net = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')

        for param in our_model.parameters():
            param.requires_grad = False
    
        self.custom_resnet_fc = torch.nn.Sequential(torch.nn.Linear(in_features=self.resnet50_full_net.fc.in_features,out_features=num_classes), 
                                torch.nn.Sigmoid())

        self.resnet50_full_net.fc = self.custom_resnet_fc

    def forward(self,x_train_mb):

        y_pred_mb = self.resnet50_full_net(x_train_mb)

        return y_pred_mb"""

In [ ]:
our_model = our_custom_resnet50_cnn(num_classes=14)
our_model = our_model.to(device)

In [ ]:
summary(our_model,input_size=(3,224,224))

In [ ]:

loss_func = torch.nn.BCELoss()
optimizer = torch.optim.SGD(params = our_model.parameters(),lr=0.001)
epochs = 5
mb_size = 10

for i in range(epochs):

    our_training_data_gen = training_data_mini_batches_generator(training_data)
    
    for x_train_mb,y_train_mb in our_training_data_gen:

        x_train_mb = torch.tensor(x_train_mb, dtype=torch.float32).permute(0, 3, 1, 2).to(device)  # Numpy -> Tensor + channel-first
        y_train_mb = torch.tensor(y_train_mb, dtype=torch.float32).to(device)


        y_pred_mb_train = our_model(x_train_mb)
        training_mb_loss = loss_func(y_pred_mb_train,y_train_mb)

        training_mb_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print("Epoch # {}, Training Loss Value = {}".format(i+1,training_mb_loss))
        